## Web scraping Wikipedia to generate S&P stock tickers of specific industry type

In [1]:
# Importing and installing required dependencies

import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

In [2]:
# !pip install pandas_datareader
from pandas_datareader import data as pdr

In [3]:
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
# !pip install --upgrade certifi
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Read HTML file and assign to variable
web_page = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [5]:
# Check variable contents
web_page

[    Symbol              Security SEC filings             GICS Sector  \
 0      MMM                    3M     reports             Industrials   
 1      AOS           A. O. Smith     reports             Industrials   
 2      ABT                Abbott     reports             Health Care   
 3     ABBV                AbbVie     reports             Health Care   
 4     ABMD               Abiomed     reports             Health Care   
 ..     ...                   ...         ...                     ...   
 498    YUM           Yum! Brands     reports  Consumer Discretionary   
 499   ZBRA    Zebra Technologies     reports  Information Technology   
 500    ZBH         Zimmer Biomet     reports             Health Care   
 501   ZION  Zions Bancorporation     reports              Financials   
 502    ZTS                Zoetis     reports             Health Care   
 
                       GICS Sub-Industry    Headquarters Location  \
 0              Industrial Conglomerates    Saint Pau

In [6]:
# Check the type and length of data stored in variable
type(web_page)

list

In [7]:
# We need the first table from the web_page variable. Check contents of first table
web_page[0]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [8]:
# Assign first table from web_page variable to another variable
company_df = web_page[0]
company_df.head(10)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
5,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
6,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
7,ADM,ADM,reports,Consumer Staples,Agricultural Products,"Chicago, Illinois",1981-07-29,7084,1902
8,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
9,ADP,ADP,reports,Information Technology,Data Processing & Outsourced Services,"Roseland, New Jersey",1981-03-31,8670,1949


In [9]:
# Check shape and data type
print(company_df.shape)
print(type(company_df))


(503, 9)
<class 'pandas.core.frame.DataFrame'>


In [10]:
#Provides descriptive statistics of all (including categorical) columns
company_df.describe(include='all')

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
count,503,503,503,503,503,503,458,5.030000e+02,503
unique,503,503,1,11,122,250,351,NaN,195
top,MMM,3M,reports,Information Technology,Health Care Equipment,"New York City, New York",1957-03-04,NaN,1985
freq,1,1,503,75,19,44,37,NaN,12
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.932141e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.535898e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000e+03,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.761050e+04,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.832410e+05,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.137954e+06,NaN


In [11]:
# Drop columns not required
company_df.drop(['SEC filings', 'Headquarters Location', 'Date first added', 'Founded', 'CIK'], axis=1, inplace=True) 
company_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ABMD,Abiomed,Health Care,Health Care Equipment


In [12]:
# Isolate all tickers
company_df['Symbol'].unique()

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE',
       'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB',
       'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK',
       'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK',
       'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX',
       'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B',
       'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL',
       'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP',
       'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD',
       'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS',
       'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', '

In [13]:
# Check industry sectors in the S&P 500 list
company_df['GICS Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Staples',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Real Estate', 'Energy'], dtype=object)

In [14]:
#Filter out only energy companies

comp_energy = company_df[(company_df['GICS Sector']=='Energy')] 
comp_energy

,Symbol,Security,GICS Sector,GICS Sub-Industry
44,APA,APA Corporation,Energy,Oil & Gas Exploration & Production
57,BKR,Baker Hughes,Energy,Oil & Gas Equipment & Services
104,CVX,Chevron Corporation,Energy,Integrated Oil & Gas
123,COP,ConocoPhillips,Energy,Oil & Gas Exploration & Production
133,CTRA,Coterra,Energy,Oil & Gas Exploration & Production
145,DVN,Devon Energy,Energy,Oil & Gas Exploration & Production
147,FANG,Diamondback Energy,Energy,Oil & Gas Exploration & Production
174,EOG,EOG Resources,Energy,Oil & Gas Exploration & Production
176,EQT,EQT,Energy,Oil & Gas Exploration & Production
190,XOM,ExxonMobil,Energy,Integrated Oil & Gas


In [15]:
# Check for tickers of energy companies
comp_energy['Symbol'].unique()

array(['APA', 'BKR', 'CVX', 'COP', 'CTRA', 'DVN', 'FANG', 'EOG', 'EQT',
       'XOM', 'HAL', 'HES', 'KMI', 'MRO', 'MPC', 'OXY', 'OKE', 'PSX',
       'PXD', 'SLB', 'VLO', 'WMB'], dtype=object)

In [16]:
# Assign start period and end period

strt_date = datetime.datetime(2020,10,15)
print(strt_date)
end_date = datetime.datetime.today()
print(end_date)

2020-10-15 00:00:00
2022-10-12 03:57:55.226423


In [17]:
#Create a new empty dataframe to write to

stk_price=pd.DataFrame()

In [18]:
# Assign downloaded data to new variable

df = yf.download('APA', start=strt_date, end=end_date)
type(df)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-10-15 00:00:00-04:00,9.100000,9.780000,8.960000,9.760000,9.571520,16482600
2020-10-16 00:00:00-04:00,9.640000,9.810000,9.340000,9.370000,9.189052,10037800
2020-10-19 00:00:00-04:00,9.450000,9.580000,9.020000,9.060000,8.885037,10772200
2020-10-20 00:00:00-04:00,9.220000,9.860000,9.180000,9.610000,9.424416,13204900
2020-10-21 00:00:00-04:00,9.410000,9.500000,9.080000,9.100000,8.947541,11784100
...,...,...,...,...,...,...
2022-10-05 00:00:00-04:00,39.099998,40.930000,38.590000,40.520000,40.520000,11925400
2022-10-06 00:00:00-04:00,40.189999,42.360001,40.090000,42.200001,42.200001,8897800
2022-10-07 00:00:00-04:00,42.160000,43.709999,41.930000,42.520000,42.520000,10916200


In [19]:
#Read and append data into the newly created dataframe

for i in comp_energy['Symbol'].unique():
    df = yf.download(i, start=strt_date, end=end_date)
        
    df['Symbol'] = i
    
    stk_price = stk_price.append(df)
    print(df.head())
    

[*********************100%***********************]  1 of 1 completed
                           Open  High   Low  Close  Adj Close    Volume Symbol
Date                                                                          
2020-10-15 00:00:00-04:00  9.10  9.78  8.96   9.76   9.571519  16482600    APA
2020-10-16 00:00:00-04:00  9.64  9.81  9.34   9.37   9.189051  10037800    APA
2020-10-19 00:00:00-04:00  9.45  9.58  9.02   9.06   8.885036  10772200    APA
2020-10-20 00:00:00-04:00  9.22  9.86  9.18   9.61   9.424416  13204900    APA
2020-10-21 00:00:00-04:00  9.41  9.50  9.08   9.10   8.947543  11784100    APA
[*********************100%***********************]  1 of 1 completed
                            Open   High    Low  Close  Adj Close    Volume  \
Date                                                                         
2020-10-15 00:00:00-04:00  12.89  13.55  12.83  13.45  12.644887   7062800   
2020-10-16 00:00:00-04:00  13.33  13.46  13.12  13.14  12.353445   7910100 

In [20]:
#Check stk_price
stk_price

,Open,High,Low,Close,Adj Close,Volume,Symbol
Date,,,,,,,
2020-10-15 00:00:00-04:00,9.100000,9.780000,8.960000,9.760000,9.571519,16482600,APA
2020-10-16 00:00:00-04:00,9.640000,9.810000,9.340000,9.370000,9.189051,10037800,APA
2020-10-19 00:00:00-04:00,9.450000,9.580000,9.020000,9.060000,8.885036,10772200,APA
2020-10-20 00:00:00-04:00,9.220000,9.860000,9.180000,9.610000,9.424416,13204900,APA
2020-10-21 00:00:00-04:00,9.410000,9.500000,9.080000,9.100000,8.947543,11784100,APA
...,...,...,...,...,...,...,...
2022-10-05 00:00:00-04:00,30.549999,30.820000,29.969999,30.459999,30.459999,7109000,WMB
2022-10-06 00:00:00-04:00,30.170000,30.590000,29.980000,30.110001,30.110001,5950400,WMB
2022-10-07 00:00:00-04:00,30.100000,30.219999,29.510000,29.790001,29.790001,7169200,WMB


In [21]:
# Descriptive stats for stk_price

stk_price.describe(include='all', datetime_is_numeric=True)

,Open,High,Low,Close,Adj Close,Volume,Symbol
count,11022.000000,11022.000000,11022.000000,11022.000000,11022.000000,1.102200e+04,11022
unique,NaN,NaN,NaN,NaN,NaN,NaN,22
top,NaN,NaN,NaN,NaN,NaN,NaN,APA
freq,NaN,NaN,NaN,NaN,NaN,NaN,501
mean,58.725970,59.774767,57.701050,58.777892,56.437232,9.765652e+06,NaN
std,44.784774,45.495047,44.041712,44.817620,42.844380,8.442910e+06,NaN
min,3.810000,3.990000,3.730000,3.850000,3.737756,4.878000e+05,NaN
25%,24.052499,24.480000,23.600000,24.059999,23.245335,3.845925e+06,NaN
50%,45.780001,47.049999,44.914999,46.000000,43.681473,7.625700e+06,NaN
75%,81.624998,83.234999,80.430002,81.930000,78.035330,1.261288e+07,NaN


In [22]:
# Choose one stock ticker to plot

df = stk_price[stk_price['Symbol'] == 'APA']
df['rolling20'] = df['Close'].rolling(window=20).mean()
df['rolling5'] = df['Close'].rolling(window=5).mean()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Open,High,Low,Close,Adj Close,Volume,Symbol,rolling20,rolling5
Date,,,,,,,,,
2020-10-15 00:00:00-04:00,9.100000,9.780000,8.960000,9.760000,9.571519,16482600,APA,NaN,NaN
2020-10-16 00:00:00-04:00,9.640000,9.810000,9.340000,9.370000,9.189051,10037800,APA,NaN,NaN
2020-10-19 00:00:00-04:00,9.450000,9.580000,9.020000,9.060000,8.885036,10772200,APA,NaN,NaN
2020-10-20 00:00:00-04:00,9.220000,9.860000,9.180000,9.610000,9.424416,13204900,APA,NaN,NaN
2020-10-21 00:00:00-04:00,9.410000,9.500000,9.080000,9.100000,8.947543,11784100,APA,NaN,9.380
...,...,...,...,...,...,...,...,...,...
2022-10-05 00:00:00-04:00,39.099998,40.930000,38.590000,40.520000,40.520000,11925400,APA,37.5795,37.224
2022-10-06 00:00:00-04:00,40.189999,42.360001,40.090000,42.200001,42.200001,8897800,APA,37.8430,38.674
2022-10-07 00:00:00-04:00,42.160000,43.709999,41.930000,42.520000,42.520000,10916200,APA,38.0645,40.340


In [23]:
# Plot candlestick chart
#Declare empty figure
fig = go.Figure()

#Add OHLC trace
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high = df['High'],
                             low = df['Low'],
                             close = df['Close'],
                             showlegend=False))

#Add moving average traces
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['rolling5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='rolling5'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['rolling20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='rolling20'))

In [24]:
#Reset the index of the DataFrame, and use the default one instead
stk_price.reset_index(inplace=True) 
stk_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2020-10-15 00:00:00-04:00,9.10,9.78,8.96,9.76,9.571519,16482600,APA
1,2020-10-16 00:00:00-04:00,9.64,9.81,9.34,9.37,9.189051,10037800,APA
2,2020-10-19 00:00:00-04:00,9.45,9.58,9.02,9.06,8.885036,10772200,APA
3,2020-10-20 00:00:00-04:00,9.22,9.86,9.18,9.61,9.424416,13204900,APA
4,2020-10-21 00:00:00-04:00,9.41,9.50,9.08,9.10,8.947543,11784100,APA


In [25]:
# Export extracted data to a csv file. Index = true to keep the index column in the output
stk_price.to_csv('stock_prices.csv', index=False)

In [63]:
# Combine into one function
def stocksprint(stocks, rolling, start_date, end_date):
  stk_price=pd.DataFrame()
  for i in stocks:
    df = yf.download(i, start=start_date, end=end_date)
    df['Symbol'] = i
    stk_price = stk_price.append(df)

    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=df.index,
                              open=df['Open'],
                              high = df['High'],
                              low = df['Low'],
                              close = df['Close'],
                              showlegend=False))
    fig.update_layout(title=i) #Add title to the chart
    
    for count in rolling:
      title = 'rolling'+str(count)

      df[title] = df['Close'].rolling(window=count).mean()

      fig.add_trace(go.Scatter(x=df.index, 
                          y=df[title], 
                          opacity=0.7, 
                          line=dict(width=2), 
                          name=title))
      
        
    fig.show()
  return stk_price
    

    

In [65]:
# Define parameters

# List of stocks
stocks = ['APA', 'TSLA']

# Time period
start_date = datetime.datetime(2020,7,4)
end_date = datetime.datetime.today()

# Set moving average window
rolling = []

stocksprint(stocks,rolling, start_date,end_date)



[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


                                 Open        High         Low       Close  \
Date                                                                        
2020-07-06 00:00:00-04:00   13.560000   13.720000   13.210000   13.530000   
2020-07-07 00:00:00-04:00   13.250000   13.380000   12.750000   12.770000   
2020-07-08 00:00:00-04:00   12.780000   13.150000   12.590000   13.070000   
2020-07-09 00:00:00-04:00   12.950000   13.100000   12.010000   12.080000   
2020-07-10 00:00:00-04:00   11.950000   12.770000   11.790000   12.710000   
...                               ...         ...         ...         ...   
2022-10-05 00:00:00-04:00  245.009995  246.669998  233.270004  240.809998   
2022-10-06 00:00:00-04:00  239.440002  244.580002  235.350006  238.130005   
2022-10-07 00:00:00-04:00  233.940002  234.570007  222.020004  223.070007   
2022-10-10 00:00:00-04:00  223.929993  226.990005  218.360001  222.960007   
2022-10-11 00:00:00-04:00  220.949997  225.750000  215.000000  216.500000   